In [11]:
import os
import pandas as pd
import tabula

In [3]:
# Set the JAVA_HOME environment variable to the Java installation directory
os.environ["JAVA_HOME"] = "/opt/homebrew/opt/openjdk/libexec/openjdk.jdk"

In [6]:
# Set display options to show all rows and columns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [7]:
%pip install -q tabula-py
%pip install requests beautifulsoup4
%pip install openpyxl
%pip install selenium
%pip install webdriver-manager

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


### **Importing**

Download the source code for the webpage of a single file.

In [8]:
import requests

# URL of the webpage
url = "https://www.ceneco.ph/generation-rate/january-2021"

# Define headers to mimic a web browser request
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
}

# Send a GET request to the URL with headers
response = requests.get(url, headers=headers)

# Check if the request was successful
if response.status_code == 200:
    # Get the HTML content of the page
    html_content = response.text

    # Save the HTML content to a file
    with open("january_2021_source.html", "w", encoding='utf-8') as file:
        file.write(html_content)
    
    print("Source code downloaded successfully!")
else:
    print(f"Failed to retrieve the page. Status code: {response.status_code}")

Source code downloaded successfully!


In [9]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta

# Create the folder for downloads if it doesn't exist
download_folder = "pdf downloads"
if not os.path.exists(download_folder):
    os.makedirs(download_folder)

# Base URL format
base_url = "https://www.ceneco.ph/generation-rate/{}-{}"

# Month and year to start from
start_date = datetime(2020, 1, 1)
end_date = datetime.today()

# Headers to mimic a real browser
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Accept-Language": "en-US,en;q=0.9",
    "Accept-Encoding": "gzip, deflate, br",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8",
    "Connection": "keep-alive",
}

# Function to download PDF from a URL
def download_pdf(pdf_url):
    pdf_response = requests.get(pdf_url, headers=headers)
    pdf_response.raise_for_status()
    file_name = os.path.join(download_folder, pdf_url.split('/')[-1])
    with open(file_name, 'wb') as file:
        file.write(pdf_response.content)
    print(f"Downloaded: {file_name}")

# Function to check for PDFs on a given URL
def check_for_pdfs(url):
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        link_tags = soup.find_all('a', href=True)
        pdf_urls = []

        for link_tag in link_tags:
            href = link_tag['href']
            if ".pdf" in href:
                pdf_url = href.split('file=')[1].split('&')[0]
                pdf_urls.append(pdf_url)

        return pdf_urls

    except requests.exceptions.HTTPError as e:
        print(f"HTTPError for URL {url}: {e}")
        return []

# Loop through the months and years
current_date = start_date

while current_date <= end_date:
    # Format the month and year for the URL
    month = current_date.strftime("%B").lower()  # e.g., 'january'
    year = current_date.strftime("%Y")  # e.g., '2020'
    
    # Generate the URL
    url = base_url.format(month, year)
    
    # Check for PDFs on the correct URL
    pdf_urls = check_for_pdfs(url)
    
    if not pdf_urls and month == 'january':
        # Handle the specific misspelling for January
        typo_url = base_url.format('janauary', year)
        pdf_urls = check_for_pdfs(typo_url)
        if pdf_urls:
            print(f"Found PDF with misspelling for January {year}.")
    
    if pdf_urls:
        for pdf_url in pdf_urls:
            download_pdf(pdf_url)
        # Move to the next month
        current_date = current_date.replace(day=28) + timedelta(days=4)
        current_date = current_date.replace(day=1)
    else:
        # Check the next 5 months
        has_valid_pdf = False
        for _ in range(5):
            current_date = current_date.replace(day=28) + timedelta(days=4)
            current_date = current_date.replace(day=1)
            if current_date > end_date:
                break
                
            next_month = current_date.strftime("%B").lower()
            next_year = current_date.strftime("%Y")
            next_url = base_url.format(next_month, next_year)
            
            next_pdf_urls = check_for_pdfs(next_url)
            if next_pdf_urls:
                has_valid_pdf = True
                break
        
        if not has_valid_pdf:
            print(f"No valid PDF found for {month.capitalize()}-{year} or the next 5 months. Ending the loop.")
            break

Downloaded: pdf downloads/January-2020.pdf
Downloaded: pdf downloads/February-2020.pdf
Downloaded: pdf downloads/March-2020.pdf
Downloaded: pdf downloads/April-2020.pdf
Downloaded: pdf downloads/May-2020.pdf
Downloaded: pdf downloads/June-2020.pdf
Downloaded: pdf downloads/July-2020.pdf
Downloaded: pdf downloads/August-2020.pdf
Downloaded: pdf downloads/September-2020.pdf
Downloaded: pdf downloads/October-2020.pdf
Downloaded: pdf downloads/November-2020.pdf
Downloaded: pdf downloads/December-2020.pdf
Downloaded: pdf downloads/January-2021-rotated-1.pdf
Downloaded: pdf downloads/February-2021-rotated-1.pdf
Downloaded: pdf downloads/Generation-Charge-for-March-2021-rotated-1.pdf
Downloaded: pdf downloads/GENERATION-CHARGE-JUNE-2021-1.pdf
Downloaded: pdf downloads/Comp.-of-Gen.-Charge-Nov.-2021.pdf
Downloaded: pdf downloads/Generation-Charge-December-2021.pdf
Downloaded: pdf downloads/Generation-Charge-January-2022.pdf
Downloaded: pdf downloads/GR-JAN.pdf
Downloaded: pdf downloads/GR-FEB.

Note: April and May 2021 are both images.

In [87]:
# Define the directory containing the PDFs
pdf_directory = "pdf downloads"

# List all files in the directory
pdf_files = [f for f in os.listdir(pdf_directory) if f.endswith('.pdf')]

# Initialize a list to store machine-readable PDFs
machine_readable_pdfs = []

# Loop through each PDF file and try to read the first page
for pdf_file in pdf_files:
    pdf_path = os.path.join(pdf_directory, pdf_file)
    try:
        # Attempt to read the first page of the PDF
        df = tabula.read_pdf(pdf_path, stream=True, pages=1)[0]
        # If successful, add the file to the list of machine-readable PDFs
        machine_readable_pdfs.append(pdf_file)
    except Exception:
        # Ignore the file if an error occurs
        continue

# Display the list of machine-readable PDFs
print("Machine-readable PDFs:")
for pdf in machine_readable_pdfs:
    print(pdf)

Machine-readable PDFs:
Generation-Charges-January-2023_compressed.pdf
Generation-Mix-November-2023_website-posting_compressed.pdf
Generation-Mix-_-April-2024.pdf
Generation-Rate-for-May-2023.pdf
Generation-charge-July-2023.pdf
Generation-charge-June-2023.pdf
Generation-Rates-March-2024.pdf
Gen-Rate-May-2024_compressed.pdf
Generation-Charge-September-2023_compressed.pdf
February-2024-Generation-Charge_compressed.pdf
Generation-Rate-February-.pdf
Generation-Rates-December-2023_compressed.pdf
July-2024-Generation-Rate.pdf
Generation-Rate-for-April-2023.pdf
Generation-Charge-December-2022_compressed.pdf
June-2024-Generation-Rate-for-CENECO-website-posting.pdf
ceneco-ceneration-rate-jan-2024.pdf
gr-aug2023.pdf
GR-OCTOBER-2023_compressed.pdf
Generation-Charge-March-2023.pdf


There are only a few machine readable files.

### **Processing**

In [157]:
import os
import pandas as pd
import tabula
import re

# Initialize an empty DataFrame to store results
big_df = pd.DataFrame()

# Function to clean the "Power Supplier" column
def clean_power_supplier(value):
    value = re.sub(r'^\d+\.\s*', '', value)  # Remove leading numbers and periods
    value = value.replace('Subtotal:', 'Subtotal')  # Remove the colon in "Subtotal:"
    value = value.replace('B. Wholesale Electricity Spot Market', 'WESM')  # Replace "B. Wholesale Electricity Spot Market" with "WESM"
    return value

# Function to extract the date from the file name
def extract_date_from_filename(filename):
    # Use regex to find the month-year pattern
    match = re.search(r'([A-Za-z]+)-(\d{4})', filename)
    if match:
        # Extract the month and year, format as "Mon-YYYY"
        month = match.group(1)[:3].capitalize()
        year = match.group(2)
        return f"{month}-{year}"
    else:
        # Check if the filename contains "February"
        if 'February' in filename:
            return "Feb-2023"
    return None

# Loop through each PDF file in the folder
folder_path = "pdf downloads"
for file_name in os.listdir(folder_path):
    if file_name.endswith(".pdf"):
        file_path = os.path.join(folder_path, file_name)
        
        try:
            # Read the PDF
            print(file_path)
            df = tabula.read_pdf(file_path, stream=True, pages=1)[0]

            # Check if the DataFrame is valid and contains necessary columns
            if df.empty:
                continue
            
            # Find columns based on the specified keywords
            columns_to_include = [
                find_column_by_keyword(df, 'Supply')[0],
                find_column_by_keyword(df, '%')[0],
                find_column_by_keyword(df, 'Purchased')[1],  # Get the second column with "Purchased"
                find_column_by_keyword(df, 'Average')[0],
            ]
            
            # Select the necessary columns
            sliced_cols_df = df[columns_to_include]

            # Rename the columns
            sliced_cols_df.columns = ["Power Supplier", "%", "kWh", "Average Generation Cost"]

            # Find the start and end indexes for slicing rows
            start_index = sliced_cols_df[sliced_cols_df.iloc[:, 0].str.contains('Supply', case=False, na=False)].index[0] + 1
            end_index = sliced_cols_df[sliced_cols_df.iloc[:, 0].str.contains('Market', case=False, na=False)].index[0]

            # Slice the DataFrame
            sliced_rows_df = sliced_cols_df.iloc[start_index:end_index + 1]

            # Exclude the row containing "Metering"
            sliced_rows_df = sliced_rows_df[~sliced_rows_df.iloc[:, 0].str.contains('Metering', case=False, na=False)]

            # Clean the first column
            sliced_rows_df.iloc[:, 0] = sliced_rows_df.iloc[:, 0].apply(clean_power_supplier)

            # Reset the index
            sliced_rows_df = sliced_rows_df.reset_index(drop=True)

            # Find the column that contains the word "Average"
            average_column = sliced_cols_df.columns[sliced_cols_df.apply(lambda col: col.str.contains('Average', case=False, na=False)).any()][0]

            # Find the row where "Generation", "Rate", and "Net" are all present
            gr_row_value = sliced_cols_df.loc[
                sliced_cols_df.iloc[:, 0].apply(
                    lambda x: isinstance(x, str) and all(word in x for word in ['Generation', 'Rate', 'Net'])
                ),
                average_column
            ].values[0]

            # Add a new column "Generation Charge" with this value
            sliced_rows_df['Generation Charge'] = gr_row_value

            # Extract the date from the filename and add as a new column
            date_value = extract_date_from_filename(file_name)
            sliced_rows_df['Date'] = date_value

            # Append the results to the big DataFrame
            big_df = pd.concat([big_df, sliced_rows_df], ignore_index=True)

        except Exception as e:
            # Skip the file if it's not machine-readable or any error occurs
            print(f"Skipping {file_name}: {e}")

big_df

pdf downloads/GR-APR.pdf
Skipping GR-APR.pdf: list index out of range
pdf downloads/Generation-Charges-January-2023_compressed.pdf
pdf downloads/GR-July.pdf
Skipping GR-July.pdf: list index out of range
pdf downloads/Generation-Mix-November-2023_website-posting_compressed.pdf
pdf downloads/Generation-Mix-_-April-2024.pdf
pdf downloads/Generation-Charge-December-2021.pdf
Skipping Generation-Charge-December-2021.pdf: list index out of range
pdf downloads/Generation-Rate-for-May-2023.pdf
pdf downloads/September-2020.pdf
Skipping September-2020.pdf: list index out of range
pdf downloads/December-2020.pdf
Skipping December-2020.pdf: list index out of range
pdf downloads/January-2021-rotated-1.pdf
Skipping January-2021-rotated-1.pdf: list index out of range
pdf downloads/Generation-charge-July-2023.pdf
pdf downloads/Generation-charge-June-2023.pdf
pdf downloads/GR-SEPTEMBER-2022.pdf
Skipping GR-SEPTEMBER-2022.pdf: list index out of range
pdf downloads/Generation-Rates-March-2024.pdf
pdf down

,Power Supplier,%,kWh,Average Generation Cost,Generation Charge,Date
0,Green Core Geothermal Inc. (GCGI),19.00%,"14,400,000",5.8798,10.0030,Jan-2023
1,Palm Concepcion Power Corp. (PCPC) (20 MW),19.00%,"14,400,000",10.5649,10.0030,Jan-2023
2,Palm Concepcion Power Corp. (PCPC),33.25%,"25,200,000",13.2972,10.0030,Jan-2023
3,CENPRI/ENERGREEN (Peaking & Reserve),0.66%,"499,800",NaN,10.0030,Jan-2023
4,Subtotal,72.43%,"54,888,879",10.8361,10.0030,Jan-2023
5,WESM,27.57%,"20,892,740",7.6804,10.0030,Jan-2023
6,Green Core Geothermal Inc. (GCGI),16.69%,"14,196,654",6.1501,7.8501,Nov-2023
7,Palm Concepcion Power Corp. (PCPC),27.54%,"23,415,000",7.5201,7.8501,Nov-2023
8,CENPRI/ENERGREEN (Peaking & Reserve),1.32%,"1,120,100",NaN,7.8501,Nov-2023
9,Subtotal,46.19%,"39,278,592",7.5988,7.8501,Nov-2023


Remove rows where the Generation Charge is NaN since these are pdfs that were not read properly and should be processed separately.

In [158]:
big_df = big_df.dropna(subset=['Generation Charge'])
big_df

,Power Supplier,%,kWh,Average Generation Cost,Generation Charge,Date
0,Green Core Geothermal Inc. (GCGI),19.00%,"14,400,000",5.8798,10.0030,Jan-2023
1,Palm Concepcion Power Corp. (PCPC) (20 MW),19.00%,"14,400,000",10.5649,10.0030,Jan-2023
2,Palm Concepcion Power Corp. (PCPC),33.25%,"25,200,000",13.2972,10.0030,Jan-2023
3,CENPRI/ENERGREEN (Peaking & Reserve),0.66%,"499,800",NaN,10.0030,Jan-2023
4,Subtotal,72.43%,"54,888,879",10.8361,10.0030,Jan-2023
5,WESM,27.57%,"20,892,740",7.6804,10.0030,Jan-2023
6,Green Core Geothermal Inc. (GCGI),16.69%,"14,196,654",6.1501,7.8501,Nov-2023
7,Palm Concepcion Power Corp. (PCPC),27.54%,"23,415,000",7.5201,7.8501,Nov-2023
8,CENPRI/ENERGREEN (Peaking & Reserve),1.32%,"1,120,100",NaN,7.8501,Nov-2023
9,Subtotal,46.19%,"39,278,592",7.5988,7.8501,Nov-2023


#### Creating Supplier Dataframe

In [160]:
unique_suppliers = big_df['Power Supplier'].unique()
unique_suppliers

array(['Green Core Geothermal Inc. (GCGI)',
       'Palm Concepcion Power Corp. (PCPC) (20 MW)',
       'Palm Concepcion Power Corp. (PCPC)',
       'CENPRI/ENERGREEN (Peaking & Reserve)', 'Subtotal', 'WESM',
       'Energy Development Corporation (EDC)'], dtype=object)

In [161]:
# Create a mapping of power suppliers to unique IDs
supplier_id_map = {supplier: id+1 for id, supplier in enumerate(unique_suppliers)}

# Create a new DataFrame from the mapping
supplier_df = pd.DataFrame(list(supplier_id_map.items()), columns=['Power Supplier', 'Power Supplier ID'])

supplier_df

,Power Supplier,Power Supplier ID
0,Green Core Geothermal Inc. (GCGI),1
1,Palm Concepcion Power Corp. (PCPC) (20 MW),2
2,Palm Concepcion Power Corp. (PCPC),3
3,CENPRI/ENERGREEN (Peaking & Reserve),4
4,Subtotal,5
5,WESM,6
6,Energy Development Corporation (EDC),7


In [162]:
# Create a mapping from Power Suppliers to Supplier IDs
supplier_mapping = dict(zip(supplier_df['Power Supplier'], supplier_df['Power Supplier ID']))

# Replace names with IDs in big_df
big_df['Power Supplier ID'] = big_df['Power Supplier'].map(supplier_mapping)

# Ensure IDs are not converted to floats
#big_df['Power Supplier ID'] = big_df['Power Supplier ID'].astype(int)

# Drop the old Power Suppliers column
big_df = big_df.drop(columns=['Power Supplier'])

big_df.head()

/var/folders/5w/f5z12zyx4tn68kmx7wk0wvv40000gn/T/ipykernel_95520/3565054818.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  big_df['Power Supplier ID'] = big_df['Power Supplier'].map(supplier_mapping)


,%,kWh,Average Generation Cost,Generation Charge,Date,Power Supplier ID
0,19.00%,"14,400,000",5.8798,10.0030,Jan-2023,1
1,19.00%,"14,400,000",10.5649,10.0030,Jan-2023,2
2,33.25%,"25,200,000",13.2972,10.0030,Jan-2023,3
3,0.66%,"499,800",NaN,10.0030,Jan-2023,4
4,72.43%,"54,888,879",10.8361,10.0030,Jan-2023,5


In [165]:
with pd.ExcelWriter("Historical_CENECO_GC_Breakdown.xlsx", engine='openpyxl') as writer:
    big_df.to_excel(writer, sheet_name='Historical GC', index=False)
    supplier_df.to_excel(writer, sheet_name='Supplier IDs', index=False)

**Future Direction**

Process files with NaN Generation Charge separately.

#### for troubleshooting loop

In [150]:
df = tabula.read_pdf("pdf downloads/Generation-charge-July-2023.pdf",stream = True, pages=1)[0]

In [151]:
df

,Unnamed: 0,Unnamed: 1,(A) (B),(C),(E= B + C),(F = E/A )
0,NaN,Average Gen Source % to Total,NaN,Other cost adjustment,NaN,NaN
1,NaN,NaN,kWh Basic Generation,"(DAA, NSS, and other",Total Gen Cost for the month,Cost for June
2,NaN,kWh,NaN,NaN,NaN,NaN
3,NaN,NaN,Purchased Cost (PhP),billing Adjustment,(PhP),2023
4,NaN,Purchased,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,(PhP),NaN,(PhP/kWh)
6,A. Power Supply Agreement (PSAs),Dispatch,NaN,NaN,NaN,NaN
7,1. Green Core Geothermal Inc. (GCGI),17.79%,"14,880,000 91,513,488.00",-,"91,513,488.00",6.1501
8,2. Palm Concepcion Power Corp. (PCPC) (20 MW),17.78%,"14,871,204 68,456,348.85",-,"68,456,348.85",4.6033
9,3. Palm Concepcion Power Corp. (PCPC),30.62%,"25,604,607 222,945,134.63",-,"222,945,134.63",8.7072


In [152]:
# Function to find the column name where a keyword is present in any of its rows
def find_column_by_keyword(df, keyword):
    mask = df.apply(lambda col: col.str.contains(keyword, case=False, na=False)).any()
    return df.columns[mask]

# Identifying the columns that contain the specified keywords in any of their rows
columns_to_include = [
    find_column_by_keyword(df, 'Supply')[0],
    find_column_by_keyword(df, '%')[0],
    find_column_by_keyword(df, 'Purchased')[1],  # Get the second column with "Purchased"
    find_column_by_keyword(df, 'Average')[0],
]

# Creating a new DataFrame with the selected columns
sliced_cols_df = df[columns_to_include]

sliced_cols_df

,Unnamed: 0,Unnamed: 1,(A) (B),Unnamed: 1
0,NaN,Average Gen Source % to Total,NaN,Average Gen Source % to Total
1,NaN,NaN,kWh Basic Generation,NaN
2,NaN,kWh,NaN,kWh
3,NaN,NaN,Purchased Cost (PhP),NaN
4,NaN,Purchased,NaN,Purchased
5,NaN,NaN,NaN,NaN
6,A. Power Supply Agreement (PSAs),Dispatch,NaN,Dispatch
7,1. Green Core Geothermal Inc. (GCGI),17.79%,"14,880,000 91,513,488.00",17.79%
8,2. Palm Concepcion Power Corp. (PCPC) (20 MW),17.78%,"14,871,204 68,456,348.85",17.78%
9,3. Palm Concepcion Power Corp. (PCPC),30.62%,"25,604,607 222,945,134.63",30.62%


In [153]:
# Assuming df is your original DataFrame

# Find the index of the row containing the word "Supply" in the first column
start_index = sliced_cols_df[sliced_cols_df.iloc[:, 0].str.contains('Supply', case=False, na=False)].index[0] + 1

# Find the index of the row containing the word "Market" in the first column
end_index = sliced_cols_df[sliced_cols_df.iloc[:, 0].str.contains('Market', case=False, na=False)].index[0]

# Slice the DataFrame to include rows between start_index and end_index (inclusive)
sliced_rows_df = sliced_cols_df.iloc[start_index:end_index + 1]

# Exclude the row that contains the word "Metering" in the first column
sliced_rows_df = sliced_rows_df[~sliced_rows_df.iloc[:, 0].str.contains('Metering', case=False, na=False)]

sliced_rows_df.columns = ["Power Supplier", "%", "kWh", "Average Generation Cost"]

sliced_rows_df

,Power Supplier,%,kWh,Average Generation Cost
7,1. Green Core Geothermal Inc. (GCGI),17.79%,"14,880,000 91,513,488.00",17.79%
8,2. Palm Concepcion Power Corp. (PCPC) (20 MW),17.78%,"14,871,204 68,456,348.85",17.78%
9,3. Palm Concepcion Power Corp. (PCPC),30.62%,"25,604,607 222,945,134.63",30.62%
10,4. CENPRI/ENERGREEN (Peaking & Reserve),1.42%,"1,191,400 39,670,552.17",1.42%
12,Subtotal:,68.15%,"56,989,521 426,382,515.47",68.15%
13,B. Wholesale Electricity Spot Market,31.85%,"26,635,570 206,988,064.67",31.85%


In [154]:
import re

# Function to clean the "Power Supplier" column
def clean_power_supplier(value):
    # Remove leading numbers and periods
    value = re.sub(r'^\d+\.\s*', '', value)
    # Remove the colon in "Subtotal:"
    value = value.replace('Subtotal:', 'Subtotal')
    # Replace "B. Wholesale Electricity Spot Market" with "WESM"
    value = value.replace('B. Wholesale Electricity Spot Market', 'WESM')
    return value

# Apply the cleaning function to the first column of sliced_df
sliced_rows_df.iloc[:, 0] = sliced_rows_df.iloc[:, 0].apply(clean_power_supplier)

sliced_rows_df = sliced_rows_df.reset_index(drop=True)
sliced_rows_df

,Power Supplier,%,kWh,Average Generation Cost
0,Green Core Geothermal Inc. (GCGI),17.79%,"14,880,000 91,513,488.00",17.79%
1,Palm Concepcion Power Corp. (PCPC) (20 MW),17.78%,"14,871,204 68,456,348.85",17.78%
2,Palm Concepcion Power Corp. (PCPC),30.62%,"25,604,607 222,945,134.63",30.62%
3,CENPRI/ENERGREEN (Peaking & Reserve),1.42%,"1,191,400 39,670,552.17",1.42%
4,Subtotal,68.15%,"56,989,521 426,382,515.47",68.15%
5,WESM,31.85%,"26,635,570 206,988,064.67",31.85%


In [155]:
# Find the column that contains the word "Average"
average_column = sliced_cols_df.columns[sliced_cols_df.apply(lambda col: col.str.contains('Average', case=False, na=False)).any()][0]

# Find the row where the first column contains the word "GR"
gr_row_value = sliced_cols_df.loc[
    sliced_cols_df.iloc[:, 0].apply(
        lambda x: isinstance(x, str) and all(word in x for word in ['Generation', 'Rate', 'Net'])
    ),
    average_column
].values[0]

print(gr_row_value)

# Add a new column "Generation Charge" with this value
sliced_rows_df['Generation Charge'] = gr_row_value

# Display the updated DataFrame
sliced_rows_df

[nan nan]


ValueError: Length of values (2) does not match length of index (6)

**Notes**

1. Need to take the Generation Rate (GR) Net